<a href="https://colab.research.google.com/github/kiran-pyt/classification-PyTorch-and-PyTorch-lightning/blob/main/pytorchlightingmodel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
from torchsummary import summary
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from pytorch_lightning import LightningModule, Trainer
from torchmetrics import Accuracy
import os

In [4]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS

1

In [5]:
dropout_value = 0.1
class Net(pl.LightningModule):
  def __init__(self,data_dir=PATH_DATASETS, hidden_size=16, learning_rate=2e-4):
    super(Net,self).__init__()

    # Set our init args as class attributes
    self.data_dir = data_dir
    self.hidden_size = hidden_size
    self.learning_rate = learning_rate

        # Hardcode some dataset specific attributes
    self.num_classes = 10
    self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )
    self.convblock1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(8),
                             nn.Dropout(dropout_value)

                             ) # output_size = 26
    self.convblock2=nn.Sequential(nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(16),
                             nn.Dropout(dropout_value)
                             ) # output_size = 24
    self.convblock3=nn.Sequential(nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(32),
                             nn.Dropout(dropout_value)
                             )  # output_size = 22

    self.pool1=nn.MaxPool2d(2,2) # output_size = 11

    self.convblock4=nn.Sequential(nn.Conv2d(in_channels=32,out_channels=8,kernel_size=(1,1),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(8),
                             nn.Dropout(dropout_value)
                             )   # output_size = 11
    self.convblock5=nn.Sequential(nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(16),
                             nn.Dropout(dropout_value)
                             )  # output_size = 9


    self.convblock6=nn.Sequential(nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(32),
                             nn.Dropout(dropout_value)
                             ) # output_size = 7
    self.convblock7=nn.Sequential(nn.Conv2d(in_channels=32,out_channels=10,kernel_size=(1,1),padding=0,bias=0),
                             #nn.ReLU(),
                             #nn.BatchNorm2d(10),
                             #nn.Dropout(dropout_value)
                             ) # output_size =
    #self.convblock8=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=10,kernel_size=(7,7),padding=0,bias=0),
                             #nn.ReLU()
                             #) # nn.ReLU() NEVER!
                               # output_size = 1 7x7x10 | 7x7x10x10 | 1x1x10

    self.gap = nn.Sequential(
           nn.AvgPool2d(kernel_size=7) # 7>> 9... nn.AdaptiveAvgPool((1, 1))2
        ) # output_size = 1


    self.accuracy = Accuracy(task='multiclass',
                                     num_classes=self.num_classes)


  def forward(self,x):
    x=self.convblock1(x)
    x=self.convblock2(x)
    x=self.convblock3(x)
    x=self.pool1(x)
    x=self.convblock4(x)
    x=self.convblock5(x)
    x=self.convblock6(x)
    x=self.convblock7(x)
    #x=self.convblock8(x)
    x = self.gap(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=-1)
###########################################################################################
  def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss
  def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss
  def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

  def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
  def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)
  def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

  def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

  def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

  def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=BATCH_SIZE, num_workers=os.cpu_count())







In [6]:
model = Net()

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [8]:
trainer = Trainer(
    accelerator ='cuda',
    max_epochs = 20,
    #progress_bar_refresh_rate=10
)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


100%|██████████| 9912422/9912422 [00:00<00:00, 173779535.72it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 42518671.05it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [9]:
trainer.test()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:149: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=19-step=4300.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=19-step=4300.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9825000166893005     │
│         val_loss          │    0.06578734517097473    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.06578734517097473, 'val_acc': 0.9825000166893005}]